# This Jupyter Notebook for VO data discovery

I experiment here with the **PyVO** library and ways to discover and download astronomical data.


Let's start by exploring **scanned photographic plates** via the **ObsCore** table and retrieving them in bulk using **DataLink** services.


In [1]:
print("Hello World!")
import pyvo

Hello World!


Connect to the GAVO TAP service

In [2]:
tap_url = "http://dc.g-vo.org/tap"
tap_service = pyvo.dal.TAPService(tap_url)
tap_service

TAPService(baseurl : 'http://dc.g-vo.org/tap', description : 'None')

Query the ObsCore table for images containing our source with specified equatorial coordinates

In [3]:
query = """
SELECT *
FROM ivoa.Obscore
WHERE dataproduct_type = 'image'
  AND CONTAINS(
        POINT('ICRS', 156.01124674021, 48.14753874736),
        s_region
      ) = 1
  AND NOT obs_collection LIKE '%RASS%'
ORDER BY t_min
"""
result = tap_service.search(query)

Convert TAPResults into a table and add ISO datetime columns
Convert 't_min' and 't_max' from MJD to ISO datetime
Save the resulting table

In [26]:
from astropy.time import Time

table = result.to_table()
table["date_min"] = Time(table['t_min'], format="mjd").isot
table["date_max"] = Time(table['t_max'], format="mjd").isot
table.write("table.txt", format="ascii", overwrite=True)
table.show_in_notebook()


/home/voz/projects/UPJS/DataDiscovery/.venv/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "d2dtf" yielded 151 of "dubious year (Note 5)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

In [34]:
# table.show_in_notebook()
# table.show_in_browser()

# tt = Time(table['t_min'], format='mjd')
table['t_min_ym'] = Time(table['t_min'], format='mjd').strftime("%Y-%m")
table['date_min','t_min_ym']
# table["scan_id"] = [obs_id.split('/')[-1] for obs_id in table['obs_id']]
table["scan_id"] = [access_url.split('/')[-1] for access_url in table['access_url']]
# table["t_min_ym", "scan_id", "obs_collection", "access_url"].write("table_dates.txt", format="ascii", overwrite=True)
table["t_min_ym", "scan_id"].write("table_dates.txt", format="ascii", overwrite=True)
# table.show_in_browser()

/home/voz/projects/UPJS/DataDiscovery/.venv/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "d2dtf" yielded 151 of "dubious year (Note 5)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


In [8]:
from pathlib import Path

def build_filename(download_dir_name, content_type, url, suffix=''):
    download_dir = Path(download_dir_name)
    download_dir.mkdir(parents=True, exist_ok=True)
    if content_type == "application/fits":
        ext = ".fits"
    elif content_type.startswith("image/"):
        ext = "." + content_type.split("/", 1)[1]
    else:
        ext = ""
    path = Path(Path(url).name + "_" + suffix)
    if ext and path.suffix.lower() != ext:
        path = path.with_suffix(ext)

    return download_dir / path



Download DataLink results for the dataset with the specified semantics

In [37]:
import requests

def download_by_semantics(datalink, suffix='', semantics='#this', download_dir_name='plates'):
    dl_list = list(datalink.bysemantics(semantics))
    if len(dl_list) == 0:
        print(datalink, semantics)
        print('is empty, skipping')
        raise ValueError("datalink table is empty, skipping")
    link = list(datalink.bysemantics(semantics))[0]
    url = link["access_url"]
    content_type = link["content_type"].lower()

    filename = build_filename(download_dir_name=download_dir_name, content_type=content_type, url=url, suffix=suffix)
    if filename.exists():
        print(f"Skipping {filename.name} (already exists)")
        return

    print(f"Downloading {filename} ...")
    r = requests.get(url)
    r.raise_for_status()  # stop if error
    with open(filename, "wb") as f:
        f.write(r.content)
    print(f"{filename} saved")


Apply *download_by_semantics* to retrieve preview images.
We attempt to download the #preview-image semantics first, expecting higher quality

In [55]:
from astropy.time import Time

for res in list(result)[:5]:
    # print(help(r))
    dl = res.getdatalink()
    jyear = Time(res["t_min"], format="mjd").jyear
    jyear_str = f'{jyear:.0f}'
    try:
        download_by_semantics(dl, semantics='#preview-image', suffix=jyear_str)
    except (requests.exceptions.HTTPError, ValueError) as e:
        print(e)
        print(f'Downloading {dl} by semantics #preview-image failed')
        try:
            download_by_semantics(dl, semantics='#preview', suffix=jyear_str)
        except requests.exceptions.HTTPError as e:
            print(f'Downloading {dl} by semantics #preview failed')


Skipping b01941_1888.jpeg (already exists)
Skipping i00601_1890.jpeg (already exists)
Skipping i00811_1890.jpeg (already exists)
Skipping i01126_1890.jpeg (already exists)
Skipping i02347_1891.jpeg (already exists)


Download #this (most likely a downsampled FITS image)

In [56]:
from astropy.time import Time

for res in list(result)[:3]:
    dl = res.getdatalink()
    jyear = Time(res["t_min"], format="mjd").jyear
    jyear_str = f'{jyear:.0f}'
    try:
        download_by_semantics(dl, semantics='#this', suffix=jyear_str)
    except (requests.exceptions.HTTPError, ValueError) as e:
        print(e)
        print(f'Downloading {dl} by semantics #this failed')


Skipping b01941_1888.fits (already exists)
Skipping i00601_1890.fits (already exists)
Skipping i00811_1890.fits (already exists)


Cut out a square region around FOI

In [35]:
from astropy.nddata import Cutout2D
help(Cutout2D)

Help on class Cutout2D in module astropy.nddata.utils:

class Cutout2D(builtins.object)
 |  Cutout2D(data, position, size, wcs=None, mode='trim', fill_value=nan, copy=False, *, limit_rounding_method=<ufunc 'ceil'>)
 |  
 |  Create a cutout object from a 2D array.
 |  
 |  The returned object will contain a 2D cutout array.  If
 |  ``copy=False`` (default), the cutout array is a view into the
 |  original ``data`` array, otherwise the cutout array will contain a
 |  copy of the original data.
 |  
 |  If a `~astropy.wcs.WCS` object is input, then the returned object
 |  will also contain a copy of the original WCS, but updated for the
 |  cutout array.
 |  
 |  For example usage, see :ref:`astropy:cutout_images`.
 |  
 |  .. warning::
 |  
 |      The cutout WCS object does not currently handle cases where the
 |      input WCS object contains distortion lookup tables described in
 |      the `FITS WCS distortion paper
 |      <https://www.atnf.csiro.au/people/mcalabre/WCS/dcs_20040422.

In [39]:
import astropy.units as u
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from pathlib import Path

ra_deg_tmp = 156.01124674021
dec_deg_tmp = 48.14753874736
radius_deg_tmp = 10/60
position = SkyCoord(ra= ra_deg_tmp* u.deg, dec= dec_deg_tmp * u.deg, frame="icrs")
size = radius_deg_tmp * 2 * u.deg
# fits_path = Path('plates/applause/HS01338_y.fits')
fits_path = Path('plates/applause/AG03170_y.fits')
with fits.open(fits_path) as hdul:
    hdu = hdul[0]
    data = hdu.data
    wcs = WCS(hdu.header)

cutout = Cutout2D(
    data,
    position=position,
    size=size,
    wcs=wcs,
    mode="trim"
    )
new_hdu = fits.PrimaryHDU(data=cutout.data, header=cutout.wcs.to_header())
out_path = fits_path.with_name(fits_path.stem + "_cut.fits")
new_hdu.writeto(out_path, overwrite=True)

Set MJD-AVG to 36305.850301 from DATE-AVG.
Set MJD-END to 36305.855498 from DATE-END'. [astropy.wcs.wcs]



Here we will touch **SODA** interface.
Quite often we do not need to download the **whole** image. Instead, we can cut out the FOI **on the server side**.

By the way, datacenters may provide a downsampled FITS file under the semantics #this, hiding the full-resolution image under the semantics #coderived. This is reasonable from the datacenter point of view, as it saves resources from occasional downloads.

However, when searching for transients or performing photometry, we do need a *high-quality* image, even though we usually do not require the *whole* plate.

Let's take a closer view at the first datalink table from our query results

In [6]:
res_first = list(result)[0]
dl = res_first.getdatalink()
t = dl.to_table()
t.show_in_notebook()
t

ID,access_url,service_def,error_message,description,semantics,content_type,content_length,local_semantics,content_qualifier
,,,,,,,byte,,
object,object,object,object,object,object,object,int64,object,object
ivo://org.gavo.dc/~?applause/q/DR3/scans/HAM-AG/AG03152_y.fits,https://www.plate-archive.org/files/DR3/scans/HAM-AG/AG03152_y.fits,,,The main data product,#this,image/fits,1037645824,,
ivo://org.gavo.dc/~?applause/q/DR3/scans/HAM-AG/AG03152_y.fits,https://www.plate-archive.org/files/DR3/previews/HAM-AGH/AG03152_pre.jpg,,,A preview or thumbnail of the data,#preview,,--,,


We are lucky: this datalink contains something with #proc semantics; **descriptions** says, that this is precisely what we need. Unfortunately, we are not able to choose cutout size time, as mentioned in the description, but this is not a case for all datalinks (indeed, some of them do not provide users with SODA interface at all)

We are lucky: this DataLink contains entries with the #proc semantics; the **description** indicates that this is exactly what we need.

Unfortunately, we cannot choose the cutout size in this case, as mentioned in the description. However, this does not relate to all DataLinks — some provide this option, while others do not offer a SODA interface at all.

To explore the SODA service, we need to know the URL of the endpoint for submitting SODA requests (where the methods reside) and the input parameters.
All of this we can extract from the DataLink table metadata and the DataLink table itself. Examining the corresponding VOTable (XML) helps to figure out where these things live.

In [57]:
[print(p) for r in dl.votable.resources for p in r.params]
from astropy.io.votable import writeto
writeto(dl.votable, "datalink.vot")


<PARAM ID="accessURL" arraysize="*" datatype="char" name="accessURL" ucd="meta.ref.url" value="http://dc.g-vo.org/dasch/q/dl/dlget"/>
<PARAM ID="standardID" arraysize="*" datatype="char" name="standardID" value="ivo://ivoa.net/std/soda#sync-1.0"/>


Take a look at the datalink. We can extract some parameters visible in the TOPCAT "Table Parameters" window this way (I hope, PyVO provides methods to handle these details under the hood). First of all, we need to extract input parameters:

In [59]:
vot = dl.votable  # pyvo.dal.adhoc.DatalinkResults VOTable
input_params_tmp = {}
# Iterate over all resources
for res in vot.resources:
    # Iterate over all GROUPs in the resource
    for group_tmp in res.groups:
        if group_tmp.ID == "inputParams":
            for item_tmp in group_tmp.iter_fields_and_params():
                print(f'{item_tmp.name=}, {item_tmp.ucd=}, {item_tmp.value=}')
                print(f'{item_tmp.description=}')
                input_params_tmp[item_tmp.name] = item_tmp.value


item.name='ID', item.ucd='meta.id;meta.main', item.value='ivo://org.gavo.dc/~?dasch/q/i00811'
item.description='The publisher DID of the dataset of interest'
item.name='POS', item.ucd='phys.angArea;obs', item.value=''
item.description='SIAv2-compatible cutout specification'


SODA endpoint is here:

In [60]:
proc_link_tmp = next(dl.bysemantics("#proc"))

Try to submit a request to the SODA

In [62]:
soda_url_tmp = proc_link_tmp.access_url
contents_type_tmp = proc_link_tmp.content_type
ra_deg_tmp = 156.01124674021
dec_deg_tmp = 48.14753874736
radius_deg_tmp = 0.5
params_tmp = {
    "ID": input_params_tmp["ID"],
    "POS": f'CIRCLE {ra_deg_tmp} {dec_deg_tmp} {radius_deg_tmp}'
}
r = requests.get(soda_url_tmp, params=params_tmp)
r

<Response [200]>

Check and save result:

In [ ]:
r.raise_for_status()
content_type_tmp = r.headers['Content-Type']
with open('tmp.tmp', "wb") as f:
    f.write(r.content)
    print(f'file is ready')

Gathering everything together:
**The Algorithm**:
1. Try to download a cutout around the **FOI** via SODA.
2. If that fails, download the full-resolution FITS image (the dataset with `#this` semantics).
3. If that fails, download the dataset with `#preview-image` semantics.
4. If that fails, download the dataset with `#preview` semantics.
5. If all else fails… well, screw it.


In [6]:
def download_cutout(dtl, ra_deg, dec_deg, radius_deg):
    proc_link = next(dtl.bysemantics("#proc"))
    input_params = {}
    for resource in dtl.votable.resources:
    # Iterate over all GROUPs in the resource
        for group in resource.groups:
            if group.ID == "inputParams":
                for item in group.iter_fields_and_params():
                    input_params[item.name] = item.value

    soda_url = proc_link.access_url
    contents_type = proc_link.content_type

    params = {
        "ID": input_params["ID"],
        "POS": f'CIRCLE {ra_deg} {dec_deg} {radius_deg}'
    }

    print(f'cutout_{ra_deg:.0f}_{dec_deg:.0f}')
    cutout_name = build_filename(download_dir_name="plates", content_type="application/fits", url=input_params["ID"], suffix=f'cutout_{ra_deg:.0f}_{dec_deg:.0f}')
    print(f"Downloading {cutout_name} ...")
    if cutout_name.exists():
        print(f"Skipping {cutout_name.name} (already exists)")
        return
    r = requests.get(soda_url, params=params)
    r.raise_for_status()
    content_type = r.headers['Content-Type']
    cutout_name = build_filename(download_dir_name="plates", content_type="application/fits", url=input_params["ID"], suffix=f'cutout_{ra_deg:.0f}_{dec_deg:.0f}')
    with open(cutout_name, "wb") as f:
        f.write(r.content)
        print(f'{cutout_name} is ready')

In [53]:
from astropy.time import Time

foi_ra_deg = 156.01124674021
foi_dec_deg = 48.14753874736
foi_radius_deg = 0.5
result_list = list(result)

for res in result:
# if True:
#     res = result_list[100]
    dl = res.getdatalink()
    t = dl.to_table()
    t.show_in_notebook()
    t
    jyear = Time(res["t_min"], format="mjd").jyear
    jyear_str = f'{jyear:.0f}'

    # Try cutout:
    try:
        download_cutout(dl, foi_ra_deg, foi_dec_deg, foi_radius_deg)
    except (requests.exceptions.HTTPError, StopIteration, ValueError):
        print("Datalink do not provide #proc")
        # Well, try to get the whole image
        print("Trying to download whole fits image by #this semantics")
        try:
            download_by_semantics(dl, semantics='#this', suffix=jyear_str)
        except (requests.exceptions.HTTPError, StopIteration, ValueError) as e:
            print(e)
            print(f'Downloading {dl} by semantics #this failed')
            # We still have alternatives
            print('Trying to download preview in the best quality with #peview-image semantics ')
            try:
                download_by_semantics(dl, semantics='#preview-image', suffix=jyear_str)
            except (requests.exceptions.HTTPError, StopIteration, ValueError) as e:
                print(e)
                print(f'Downloading {dl} by semantics #preview-image failed')
                # I doubt we will be happy with thumbnails, but seems, this is a last chance
                try:
                    download_by_semantics(dl, semantics='#preview', suffix=jyear_str)
                except (requests.exceptions.HTTPError, StopIteration, ValueError) as e:
                    print(f'Downloading {dl} by semantics #preview failed')
                    # Well, to hell with it, let's move on
                    # continue


cutout_156_48
Skipping b01941_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i00601_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i00811_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i01126_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i02347_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i05998_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i05999_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i06043_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i06162_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i08266_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i08359_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i08382_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i08395_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i12127_cutout_156_48.fits (already exists)
cutout_156_48
Skipping i12306_cutout_156_48.fits (already exists)
cutout_156